In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [2]:
with open('dotu.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)
print(vocabulary_size)

['\t', '\n', '\x1e', ' ', '!', '%', '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '©', '«', '±', '»', 'Ё', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '—', '“', '”', '…', '№']
152


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('Управление')
decoded_hello = decode(encoded_hello)

print(encoded_hello)
print(decoded_hello)

[101, 129, 130, 114, 116, 125, 119, 127, 122, 119]
Управление


In [4]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([  3,   3,   1,  86, 128, 131, 132, 114, 132, 128, 137, 127, 128,   3,
        128, 115, 139, 114, 145,   3, 132, 119, 128, 130, 122, 145,   3, 133,
        129, 130, 114, 116, 125, 119, 127, 122, 145,   1,  51,  51,  51,  51,
         51,  51,  51,  51,  51,  51,  51,  51,  51,  51,   1,  97, 128, 131,
        132, 114, 127, 128, 116, 128, 137, 127, 141, 119,   3, 126, 114, 132,
        119, 130, 122, 114, 125, 141,   3, 133, 137, 119, 115, 127, 128, 117,
        128,   3, 124, 133, 130, 131, 114,   3,   1, 134, 114, 124, 133, 125,
        142, 132])


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([3]) target is tensor(3)
when input is tensor([3, 3]) target is tensor(1)
when input is tensor([3, 3, 1]) target is tensor(86)
when input is tensor([ 3,  3,  1, 86]) target is tensor(128)
when input is tensor([  3,   3,   1,  86, 128]) target is tensor(131)
when input is tensor([  3,   3,   1,  86, 128, 131]) target is tensor(132)
when input is tensor([  3,   3,   1,  86, 128, 131, 132]) target is tensor(114)
when input is tensor([  3,   3,   1,  86, 128, 131, 132, 114]) target is tensor(132)
